In [260]:
import pandas as pd
import os

In [262]:
directory_path = 'C:/Users/matte/Desktop/Epicode/Capston project/dataset/'
csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
dataframes = []

for file in csv_files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_csv(file_path)
    
    ticker_id = file.split()[0]
    df['ticker_id'] = ticker_id

    df['Data'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')
    df = df.sort_values(by='Data')
    df['Data'] = df['Data'].dt.strftime('%d/%m/%Y')

    df['Var. %'] = df['Var. %'].str.replace('%', '').str.replace(',', '.').astype(float)
    
    cols_to_convert = ['Ultimo', 'Apertura', 'Massimo', 'Minimo']
    for col in cols_to_convert:
            df[col] = df[col].str.replace('.', '', regex=False).str.replace(',', '.').astype(float)
    
    initial_investment = 1000
    df['Investment'] = initial_investment * (1 + df['Var. %'] / 100).cumprod()

    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)
combined_df['Data'] = pd.to_datetime(combined_df['Data'], format='%d/%m/%Y')


In [264]:
combined_df.dtypes

Data          datetime64[ns]
Ultimo               float64
Apertura             float64
Massimo              float64
Minimo               float64
Vol.                  object
Var. %               float64
ticker_id             object
Investment           float64
dtype: object

In [266]:
filtered_df = combined_df[~combined_df['ticker_id'].isin(['BTC', 'XAU'])].copy()
unique_dates = filtered_df['Data'].unique()
mean_investment = filtered_df.groupby('Data')['Investment'].mean().reset_index()
mean_investment = mean_investment.sort_values(by='Data').copy()
mean_investment['Var. %'] = mean_investment['Investment'].pct_change() * 100
initial_investment = 1000
mean_investment.loc[0, 'Var. %'] = (mean_investment.loc[0, 'Investment'] - initial_investment) / initial_investment * 100
mean_investment['ticker_id'] = 'ALL_S'
all_stock_df = mean_investment[['Data', 'Var. %', 'ticker_id', 'Investment']]
combined_df = pd.concat([combined_df, all_stock_df], ignore_index=True)

In [268]:
filtered_low_r = combined_df[combined_df['ticker_id'].isin(['ALL_S', 'BTC_EUR', 'XAU_EUR'])].copy()
filtered_low_r = filtered_low_r.sort_values(by='Data')
results = []
grouped = filtered_low_r.groupby('Data')
for date, group in grouped:
    all_s_value = group[group['ticker_id'] == 'ALL_S']['Investment'].values[0]
    btc_value = group[group['ticker_id'] == 'BTC_EUR']['Investment'].values[0]
    xau_value = group[group['ticker_id'] == 'XAU_EUR']['Investment'].values[0]
    investment = (all_s_value * 0.30) + (btc_value * 0.10) + (xau_value * 0.60)
    results.append({'Data': date, 'Investment': investment})
weighted_investment_df = pd.DataFrame(results)
weighted_investment_df['Var. %'] = weighted_investment_df['Investment'].pct_change() * 100
initial_investment = 1000
weighted_investment_df.loc[0, 'Var. %'] = (weighted_investment_df.loc[0, 'Investment'] - initial_investment) / initial_investment * 100
weighted_investment_df['ticker_id'] = 'LOW_R'
low_r_df = weighted_investment_df[['Data', 'Var. %', 'ticker_id', 'Investment']]
combined_df = pd.concat([combined_df, low_r_df], ignore_index=True)

In [269]:
filtered_mid_r = combined_df[combined_df['ticker_id'].isin(['ALL_S', 'BTC_EUR', 'XAU_EUR'])].copy()
filtered_mid_r = filtered_mid_r.sort_values(by='Data')
results_m = []
grouped_m = filtered_mid_r.groupby('Data')
for date, group in grouped_m:
    all_s_value = group[group['ticker_id'] == 'ALL_S']['Investment'].values[0]
    btc_value = group[group['ticker_id'] == 'BTC_EUR']['Investment'].values[0]
    xau_value = group[group['ticker_id'] == 'XAU_EUR']['Investment'].values[0]
    investment = (all_s_value * 0.60) + (btc_value * 0.20) + (xau_value * 0.20)
    results_m.append({'Data': date, 'Investment': investment})
weighted_m_investment_df = pd.DataFrame(results_m)
weighted_m_investment_df['Var. %'] = weighted_m_investment_df['Investment'].pct_change() * 100
initial_investment = 1000
weighted_m_investment_df.loc[0, 'Var. %'] = (weighted_m_investment_df.loc[0, 'Investment'] - initial_investment) / initial_investment * 100
weighted_m_investment_df['ticker_id'] = 'MID_R'
mid_r_df = weighted_m_investment_df[['Data', 'Var. %', 'ticker_id', 'Investment']]
combined_df = pd.concat([combined_df, mid_r_df], ignore_index=True)

In [272]:
filtered_high_r = combined_df[combined_df['ticker_id'].isin(['ALL_S', 'BTC_EUR', 'XAU_EUR'])].copy()
filtered_high_r = filtered_high_r.sort_values(by='Data')
results_h = []
grouped_h = filtered_high_r.groupby('Data')
for date, group in grouped_h:
    all_s_value = group[group['ticker_id'] == 'ALL_S']['Investment'].values[0]
    btc_value = group[group['ticker_id'] == 'BTC_EUR']['Investment'].values[0]
    xau_value = group[group['ticker_id'] == 'XAU_EUR']['Investment'].values[0]
    investment = (all_s_value * 0.30) + (btc_value * 0.60) + (xau_value * 0.10)
    results_h.append({'Data': date, 'Investment': investment})
weighted_h_investment_df = pd.DataFrame(results_h)
weighted_h_investment_df['Var. %'] = weighted_h_investment_df['Investment'].pct_change() * 100
initial_investment = 1000
weighted_h_investment_df.loc[0, 'Var. %'] = (weighted_h_investment_df.loc[0, 'Investment'] - initial_investment) / initial_investment * 100
weighted_h_investment_df['ticker_id'] = 'HIGH_R'
high_r_df = weighted_h_investment_df[['Data', 'Var. %', 'ticker_id', 'Investment']]
combined_df = pd.concat([combined_df, high_r_df], ignore_index=True)

In [273]:
combined_df['Investment'] = combined_df['Investment'].round(2)
combined_df['Var. %'] = combined_df['Var. %'].round(2)

In [282]:
combined_df.sample(60)

,Data,Ultimo,Apertura,Massimo,Minimo,Vol.,Var. %,ticker_id,Investment
2314,2019-11-01,5.8300,5.9440,6.0340,5.7140,"125,86M",-1.59,TRN,1066.55
1160,2021-02-01,4.8820,4.9660,5.2200,4.8820,"45,52M",-1.41,IG,860.98
2141,2020-06-01,24.2200,22.5000,25.3200,21.8500,"71,59M",9.64,STMMI,1768.77
579,2022-09-01,9.1120,9.7500,9.8620,8.7820,"35,21M",-7.10,CPRI,1042.02
826,2023-04-01,6.1990,5.6340,6.2570,5.5650,"546,68M",10.17,ENEI,1113.06
1922,2022-03-01,45.6400,44.0000,47.3200,40.3300,"6,48M",4.30,RECI,1233.65
840,2019-06-01,14.6060,13.4160,14.7140,13.3880,"229,54M",7.56,ENI,1075.60
1104,2021-06-01,3.4840,3.4990,3.7690,3.4620,"78,06M",-0.20,HRA,1072.22
123,2019-09-01,16.9740,15.8210,17.3780,15.8060,"22,50M",7.46,AZMT,1157.72
264,2021-06-01,49.3200,51.9000,52.7500,46.9200,"1,63M",-4.33,BCU,1751.16


In [286]:
file_path = 'C:/Users/matte/Desktop/Epicode/Capston project/Dataset finale.csv'
combined_df.to_csv(file_path, index=False)

In [288]:
category_data = [
    {"ticker_id": "CRDI", "name": "UniCredit SpA", "category": "Financials"},
    {"ticker_id": "ISP", "name": "Intesa Sanpaolo SpA", "category": "Financials"},
    {"ticker_id": "GASI", "name": "Assicurazioni Generali SpA", "category": "Financials"},
    {"ticker_id": "FBK", "name": "FinecoBank Banca Fineco SpA", "category": "Financials"},
    {"ticker_id": "MDBI", "name": "Mediobanca Banca di Credito Finanziario SpA", "category": "Financials"},
    {"ticker_id": "BAMI", "name": "Banco Bpm SpA", "category": "Financials"},
    {"ticker_id": "EMII", "name": "Bper Banca SpA", "category": "Financials"},
    {"ticker_id": "BPSI", "name": "Banca Popolare di Sondrio ScpA", "category": "Financials"},
    {"ticker_id": "BMED", "name": "Banca Mediolanum SpA", "category": "Financials"},
    {"ticker_id": "BMPS", "name": "Banca Monte dei Paschi di Siena SpA", "category": "Financials"},
    {"ticker_id": "AZMT", "name": "Azimut Holding SpA", "category": "Financials"},
    {"ticker_id": "UNPI", "name": "Unipol Gruppo Finanziario Spa", "category": "Financials"},
    {"ticker_id": "PST", "name": "Poste Italiane SpA", "category": "Financials"},
    {"ticker_id": "MONC", "name": "Moncler SpA", "category": "Consumer Goods"},
    {"ticker_id": "RACE", "name": "Ferrari NV", "category": "Consumer Goods"},
    {"ticker_id": "STLAM", "name": "Stellantis NV", "category": "Consumer Goods"},
    {"ticker_id": "PIRC", "name": "Pirelli & C SPA", "category": "Consumer Goods"},
    {"ticker_id": "BCU", "name": "Brunello Cucinelli SpA", "category": "Consumer Goods"},
    {"ticker_id": "CPRI", "name": "Davide Campari Milano SpA", "category": "Consumer Goods"},
    {"ticker_id": "AMPF", "name": "Amplifon SpA", "category": "Health Care"},
    {"ticker_id": "RECI", "name": "Recordati", "category": "Health Care"},
    {"ticker_id": "DIAS", "name": "DiaSorin SpA", "category": "Health Care"},
    {"ticker_id": "LDOF", "name": "Leonardo SpA", "category": "Industrials"},
    {"ticker_id": "PRY", "name": "Prysmian SpA", "category": "Industrials"},
    {"ticker_id": "IVG", "name": "Iveco Group NV", "category": "Industrials"},
    {"ticker_id": "ITPG", "name": "Interpump Group", "category": "Industrials"},
    {"ticker_id": "NEXII", "name": "Nexi SpA", "category": "Industrials"},
    {"ticker_id": "TENR", "name": "Tenaris SA", "category": "Energy"},
    {"ticker_id": "ENI", "name": "Eni SpA", "category": "Energy"},
    {"ticker_id": "SRG", "name": "Snam SpA", "category": "Energy"},
    {"ticker_id": "SPMI", "name": "Saipem SpA", "category": "Energy"},
    {"ticker_id": "STMMI", "name": "STMicroelectronics NV", "category": "Technology"},
    {"ticker_id": "INWT", "name": "Inwit", "category": "Technology"},
    {"ticker_id": "TLIT", "name": "Telecom Italia", "category": "Technology"},
    {"ticker_id": "IG", "name": "Italgas SpA", "category": "Utilities"},
    {"ticker_id": "TRN", "name": "Terna Rete Elettrica Nazionale SpA", "category": "Utilities"},
    {"ticker_id": "ENEI", "name": "Enel", "category": "Utilities"},
    {"ticker_id": "ERG", "name": "ERG SpA", "category": "Utilities"},
    {"ticker_id": "HRA", "name": "Hera SpA", "category": "Utilities"},
    {"ticker_id": "A2", "name": "A2A SpA", "category": "Utilities"},
    {"ticker_id": "BTC_EUR", "name": "Bitcoin", "category": "Cryptocurrency"},
    {"ticker_id": "XAU_EUR", "name": "Oro", "category": "Precious Metal"},
    {"ticker_id": "ALL_S", "name": "Aziende MIB media", "category": "Index"},
    {"ticker_id": "LOW_R", "name": "Portfolio basso rischio", "category": "Portfolio"},
    {"ticker_id": "MID_R", "name": "Portfolio medio rischio", "category": "Portfolio"},
    {"ticker_id": "HIGH_R", "name": "Portfolio alto rischio", "category": "Portfolio"}
]
category_df = pd.DataFrame(category_data)
file_path = 'C:/Users/matte/Desktop/Epicode/Capston project/Categorie.csv'
category_df.to_csv(file_path, index=False)